In [1]:
!pip install matplotlib google-cloud-storage

In [2]:
import collections
import copy
import hashlib
import io
import os
import subprocess
import textwrap
import time
import glob

from typing import List, Text

from PIL import Image

import cv2
import numpy as np
import pandas as pd
import tabulate
import seaborn as sns
from tqdm import tqdm
import ast


import tensorflow as tf

import matplotlib.pyplot as plt

In [3]:
from google.colab import auth

# Authenticate user for access. There will be a popup asking you to sign in with your user and approve access.
auth.authenticate_user()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#@title Global params

class Globals:
  # GCP project with GCS bucket of interest
  gcp_project = 'dx-scin-public' #@param

  # GCS bucket with data to read
  gcs_bucket_name = 'dx-scin-public-data' #@param

  # CSV of case metadata to read
  cases_csv = 'dataset/scin_cases.csv' #@param

  # CSV of label metadata to read
  labels_csv = 'dataset/scin_labels.csv' #@param

  # Images directory
  gcs_images_dir = 'dataset/images/' #@param

  ### Key column names
  image_path_columns = ['image_1_path', 'image_2_path', 'image_3_path']
  weighted_skin_condition_label = "weighted_skin_condition_label"
  skin_condition_label = "dermatologist_skin_condition_on_label_name"

  ###### Formed during execution:

  # Client for querying GCS
  gcs_storage_client = None

  # Bucket object for loading files
  gcs_bucket = None

  # pd.DataFrame for the loaded metadata_csv
  cases_df = None

  # pd.DataFrame for the loaded labels_csv
  cases_and_labels_df = None

print(f'GCS bucket name: {Globals.gcs_bucket_name}')
print(f'cases_csv: {Globals.cases_csv}')
print(f'labels_csv: {Globals.labels_csv}')
print(f'images dir: {Globals.gcs_images_dir}')

GCS bucket name: dx-scin-public-data
cases_csv: dataset/scin_cases.csv
labels_csv: dataset/scin_labels.csv
images dir: dataset/images/


## Load dataset

In [6]:
#@title Create a dataframe that contains the metadata and condition labels

from google.cloud import storage

def list_blobs(storage_client, bucket_name):
  """Helper to list blobs in a bucket (useful for debugging)."""
  blobs = storage_client.list_blobs(bucket_name)
  for blob in blobs:
    print(blob)

def initialize_df_with_metadata(bucket, csv_path):
  """Loads the given CSV into a pd.DataFrame."""
  df = pd.read_csv(io.BytesIO(bucket.blob(csv_path).download_as_string()), dtype={'case_id': str})
  df['case_id'] = df['case_id'].astype(str)
  return df

def augment_metadata_with_labels(df, bucket, csv_path):
  """Loads the given CSV into a pd.DataFrame."""
  labels_df = pd.read_csv(io.BytesIO(bucket.blob(csv_path).download_as_string()), dtype={'case_id': str})
  labels_df['case_id'] = labels_df['case_id'].astype(str)
  merged_df = pd.merge(df, labels_df, on='case_id')
  return merged_df

Globals.gcs_storage_client = storage.Client(Globals.gcp_project)
Globals.gcs_bucket = Globals.gcs_storage_client.bucket(
    Globals.gcs_bucket_name
)
Globals.cases_df = initialize_df_with_metadata(Globals.gcs_bucket, Globals.cases_csv)
Globals.cases_and_labels_df = augment_metadata_with_labels(Globals.cases_df, Globals.gcs_bucket, Globals.labels_csv)
print(len(Globals.cases_and_labels_df))

5033


In [7]:
Globals.cases_and_labels_df.columns

Index(['case_id', 'source', 'release', 'year', 'age_group', 'sex_at_birth',
       'fitzpatrick_skin_type',
       'race_ethnicity_american_indian_or_alaska_native',
       'race_ethnicity_asian', 'race_ethnicity_black_or_african_american',
       'race_ethnicity_hispanic_latino_or_spanish_origin',
       'race_ethnicity_middle_eastern_or_north_african',
       'race_ethnicity_native_hawaiian_or_pacific_islander',
       'race_ethnicity_white', 'race_ethnicity_other_race',
       'race_ethnicity_prefer_not_to_answer', 'textures_raised_or_bumpy',
       'textures_flat', 'textures_rough_or_flaky', 'textures_fluid_filled',
       'body_parts_head_or_neck', 'body_parts_arm', 'body_parts_palm',
       'body_parts_back_of_hand', 'body_parts_torso_front',
       'body_parts_torso_back', 'body_parts_genitalia_or_groin',
       'body_parts_buttocks', 'body_parts_leg', 'body_parts_foot_top_or_side',
       'body_parts_foot_sole', 'body_parts_other',
       'condition_symptoms_bothersome_appearan

In [8]:
Globals.cases_and_labels_df.sample(1)

,case_id,source,release,year,age_group,sex_at_birth,fitzpatrick_skin_type,race_ethnicity_american_indian_or_alaska_native,race_ethnicity_asian,race_ethnicity_black_or_african_american,...,dermatologist_gradable_for_fitzpatrick_skin_type_1,dermatologist_gradable_for_fitzpatrick_skin_type_2,dermatologist_gradable_for_fitzpatrick_skin_type_3,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,gradable_for_monk_skin_tone_india,gradable_for_monk_skin_tone_us,monk_skin_tone_label_india,monk_skin_tone_label_us
513,-2683650026574626295,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NaN,NaN,NaN,NaN,...,YES,NaN,NaN,FST3,NaN,NaN,True,True,2.0,3.0


## Important Functions

**Variables**
* df_original: The full, unmodified dataset containing all cases, metadata, and image paths. This is equivalent to Globals.cases_and_labels_df and can be used like a normal pandas DataFrame.
* df_filtered: A working copy of the dataset that you can safely modify, filter, or clean without affecting the original.
* image_dir : Use this directory to access images within google cloud


**Functions**
* read_image_from_gcs(gcs_path)
  - Downloads and decodes an image directly from your GCS bucket using the path stored in the dataset (e.g. "dataset/images/12345.png").
*  get_all_image_paths(df):
  - Extracts all unique image paths from the three image columns (image_1_path, image_2_path, image_3_path) in the dataset.
* show_case_images(case_id)
  - Displays all available images for a given case_id directly from GCS.
* convert_to_binary_var(col_name)
  - convert to binary values

  Note: add instructions if we want to analyze/change/decode all images together

In [9]:
df_original = Globals.cases_and_labels_df

In [10]:
df_filtered = df_original.copy()

In [11]:
image_dir = "/content/gcs_mount/dataset/images"

In [12]:
def read_image_from_gcs(gcs_path):
    """Reads and decodes an image from GCS (relative path)."""
    try:
        blob = Globals.gcs_bucket.blob(gcs_path)
        img_bytes = blob.download_as_bytes()
        img_array = np.frombuffer(img_bytes, np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        if img is None:
            return None
        # Convert BGR (OpenCV) → RGB (matplotlib)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img_rgb
    except Exception as e:
        print(f"Failed to read {gcs_path}: {e}")
        return None


In [13]:
def get_all_image_paths(df):
    """
    Flatten all image_1/2/3 columns from the given DataFrame
    into a unique list of image paths.
    """
    image_cols = ['image_1_path', 'image_2_path', 'image_3_path']

    all_paths = (
        df[image_cols]
        .stack()          # combine all image columns vertically
        .dropna()         # remove missing entries
        .unique()         # only keep unique paths
        .tolist()         # convert to list
    )

    print(f"📸 Found {len(all_paths)} unique image paths.")
    return all_paths


In [14]:
def show_case_images(case_id):
    """Display all available images for a given case_id."""
    row = Globals.cases_and_labels_df[Globals.cases_and_labels_df["case_id"] == str(case_id)]
    if row.empty:
        print(f"No case found for ID {case_id}")
        return

    paths = row[['image_1_path', 'image_2_path', 'image_3_path']].dropna(axis=1).values.flatten()
    plt.figure(figsize=(15, 5))
    for i, path in enumerate(paths, 1):
        img = read_image_from_gcs(path)
        plt.subplot(1, len(paths), i)
        plt.imshow(img)
        plt.title(f"{os.path.basename(path)}", fontsize=9)
        plt.axis("off")
    plt.suptitle(f"Case ID: {case_id}", fontsize=12)
    plt.tight_layout()
    plt.show()


In [15]:
def convert_to_binary_var(col_name):
  df_filtered[col_name].replace('YES', 1, inplace = True)
  df_filtered[col_name].fillna(0, inplace = True)
  print(df_filtered[col_name].value_counts())
  print("")


## No longer need to run this -- Feature Cleaning, Preprocessing, and Exploration


Note: Out of 5033 cases, there are 143 problematic rows where the weight equals to above 1.0 with few cases of 0.99 (cross checked with original excel sheet)

In [ ]:
# df_filtered['fitzpatrick_skin_type'].fillna('NONE_IDENTIFIED', inplace = True)
# print(df_filtered['fitzpatrick_skin_type='].count())
# df_filtered['fitzpatrick_skin_type'].value_counts()


In [ ]:
'''
race_ethnicity_cols_list = ['race_ethnicity_american_indian_or_alaska_native', 'race_ethnicity_asian', 'race_ethnicity_black_or_african_american', 'race_ethnicity_hispanic_latino_or_spanish_origin', 'race_ethnicity_middle_eastern_or_north_african', 'race_ethnicity_native_hawaiian_or_pacific_islander', 'race_ethnicity_white', 'race_ethnicity_other_race', 'race_ethnicity_prefer_not_to_answer', 'race_ethnicity_two_or_more_after_mitigation']

for col in race_ethnicity_cols_list:
  convert_to_binary_var(col)
'''

"\nrace_ethnicity_cols_list = ['race_ethnicity_american_indian_or_alaska_native', 'race_ethnicity_asian', 'race_ethnicity_black_or_african_american', 'race_ethnicity_hispanic_latino_or_spanish_origin', 'race_ethnicity_middle_eastern_or_north_african', 'race_ethnicity_native_hawaiian_or_pacific_islander', 'race_ethnicity_white', 'race_ethnicity_other_race', 'race_ethnicity_prefer_not_to_answer', 'race_ethnicity_two_or_more_after_mitigation']\n\nfor col in race_ethnicity_cols_list:\n  convert_to_binary_var(col)\n"

In [ ]:
'''
texture_cols_list = ['textures_raised_or_bumpy', 'textures_flat', 'textures_rough_or_flaky', 'textures_fluid_filled']

for col in texture_cols_list:
  convert_to_binary_var(col)
'''

"\ntexture_cols_list = ['textures_raised_or_bumpy', 'textures_flat', 'textures_rough_or_flaky', 'textures_fluid_filled']\n\nfor col in texture_cols_list:\n  convert_to_binary_var(col)\n"

In [ ]:
'''
body_parts_col_start = df_original.columns.get_loc('body_parts_head_or_neck')
body_parts_col_end = df_original.columns.get_loc('body_parts_other')
body_parts_cols_list = df_filtered.iloc[:, body_parts_col_start:body_parts_col_end + 1 ]

for col in body_parts_cols_list:
  convert_to_binary_var(col)
'''

"\nbody_parts_col_start = df_original.columns.get_loc('body_parts_head_or_neck')\nbody_parts_col_end = df_original.columns.get_loc('body_parts_other')\nbody_parts_cols_list = df_filtered.iloc[:, body_parts_col_start:body_parts_col_end + 1 ]\n\nfor col in body_parts_cols_list:\n  convert_to_binary_var(col)\n"

In [ ]:
'''
symptoms_col_start = df_original.columns.get_loc('condition_symptoms_bothersome_appearance')
symptoms_col_end = df_original.columns.get_loc('other_symptoms_no_relevant_symptoms')
symptoms_cols_list = df_filtered.iloc[:, symptoms_col_start:symptoms_col_end + 1 ]

for col in symptoms_cols_list:
  convert_to_binary_var(col)
'''

"\nsymptoms_col_start = df_original.columns.get_loc('condition_symptoms_bothersome_appearance')\nsymptoms_col_end = df_original.columns.get_loc('other_symptoms_no_relevant_symptoms')\nsymptoms_cols_list = df_filtered.iloc[:, symptoms_col_start:symptoms_col_end + 1 ]\n\nfor col in symptoms_cols_list:\n  convert_to_binary_var(col)\n"

In [ ]:
'''
df_filtered['related_category'].fillna("NO_RESPONSE", inplace=True)
df_filtered['condition_duration'].fillna("NO_RESPONSE", inplace=True)
df_filtered['image_2_shot_type'].fillna("N/A", inplace=True)
df_filtered['image_3_shot_type'].fillna("N/A", inplace=True)
df_filtered['combined_race'].fillna("NO_RESPONSE", inplace=True)
'''

'\ndf_filtered[\'related_category\'].fillna("NO_RESPONSE", inplace=True)\ndf_filtered[\'condition_duration\'].fillna("NO_RESPONSE", inplace=True)\ndf_filtered[\'image_2_shot_type\'].fillna("N/A", inplace=True)\ndf_filtered[\'image_3_shot_type\'].fillna("N/A", inplace=True)\ndf_filtered[\'combined_race\'].fillna("NO_RESPONSE", inplace=True)\n'

In [ ]:
'''
df_filtered = df_filtered[
    ~(
        (df_filtered['dermatologist_skin_condition_on_label_name'].isna() | (df_filtered['dermatologist_skin_condition_on_label_name'].astype(str) == '[]')) &
        (df_filtered['dermatologist_skin_condition_confidence'].isna() | (df_filtered['dermatologist_skin_condition_confidence'].astype(str) == '[]'))
    )
]
print(len(df_filtered))
'''

"\ndf_filtered = df_filtered[\n    ~(\n        (df_filtered['dermatologist_skin_condition_on_label_name'].isna() | (df_filtered['dermatologist_skin_condition_on_label_name'].astype(str) == '[]')) &\n        (df_filtered['dermatologist_skin_condition_confidence'].isna() | (df_filtered['dermatologist_skin_condition_confidence'].astype(str) == '[]'))\n    )\n]\nprint(len(df_filtered))\n"

In [ ]:
'''
print(df_filtered['dermatologist_skin_condition_on_label_name'].value_counts()[:30])
print(df_filtered['dermatologist_skin_condition_confidence'].value_counts()[:30])
'''

"\nprint(df_filtered['dermatologist_skin_condition_on_label_name'].value_counts()[:30])\nprint(df_filtered['dermatologist_skin_condition_confidence'].value_counts()[:30])\n"

In [ ]:
'''

# Create a temporary DataFrame for this check, working with df_original
temp_df = df_original.copy()

# Convert stringified dictionaries in 'weighted_skin_condition_label' to actual dicts
temp_df['parsed_weights'] = temp_df['weighted_skin_condition_label'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Calculate the sum of weights for each case_id from the parsed dictionary
def sum_dict_values(weight_dict):
    if isinstance(weight_dict, dict):
        # Sum only numeric values, ignore None if present in dictionary values
        return sum(v for v in weight_dict.values() if isinstance(v, (int, float)))
    return np.nan # Return NaN if not a dictionary or empty

temp_df['original_weights_sum'] = temp_df['parsed_weights'].apply(sum_dict_values)

# Identify cases where the sum is not approximately 1 (ignoring NaNs)
original_problematic_cases = temp_df[
    ~np.isclose(temp_df['original_weights_sum'], 1.0, equal_nan=True)
]

if original_problematic_cases.empty:
    print("All original 'weighted_skin_condition_label' dictionaries sum approximately to 1.")
else:
    print("Case IDs where original 'weighted_skin_condition_label' sums do NOT approximate 1:")
    # Display case_id, original dictionary, and its sum
    display(original_problematic_cases[['case_id', 'weighted_skin_condition_label', 'original_weights_sum']])
'''

'\n\n# Create a temporary DataFrame for this check, working with df_original\ntemp_df = df_original.copy()\n\n# Convert stringified dictionaries in \'weighted_skin_condition_label\' to actual dicts\ntemp_df[\'parsed_weights\'] = temp_df[\'weighted_skin_condition_label\'].apply(\n    lambda x: ast.literal_eval(x) if isinstance(x, str) else x\n)\n\n# Calculate the sum of weights for each case_id from the parsed dictionary\ndef sum_dict_values(weight_dict):\n    if isinstance(weight_dict, dict):\n        # Sum only numeric values, ignore None if present in dictionary values\n        return sum(v for v in weight_dict.values() if isinstance(v, (int, float)))\n    return np.nan # Return NaN if not a dictionary or empty\n\ntemp_df[\'original_weights_sum\'] = temp_df[\'parsed_weights\'].apply(sum_dict_values)\n\n# Identify cases where the sum is not approximately 1 (ignoring NaNs)\noriginal_problematic_cases = temp_df[\n    ~np.isclose(temp_df[\'original_weights_sum\'], 1.0, equal_nan=True)\n]

In [ ]:
# Assuming 'problematic_cases' from the previous output is still available
'''
if not problematic_cases.empty:
    print("Counts of each problematic sum value:")
    display(problematic_cases.value_counts().sort_index())
else:
    print("No problematic cases to summarize.")
'''

'\nif not problematic_cases.empty:\n    print("Counts of each problematic sum value:")\n    display(problematic_cases.value_counts().sort_index())\nelse:\n    print("No problematic cases to summarize.")\n'

In [ ]:
# Assuming 'problematic_cases' is available from the previous step
'''
if not problematic_cases.empty:
    # Filter problematic_cases for sums greater than 1.01
    cases_gt_1_01 = problematic_cases[problematic_cases > 1.01]

    print(f"Number of cases where condition_weight sum is greater than 1.01: {len(cases_gt_1_01)}")
    if not cases_gt_1_01.empty:
        print("\nCounts of each sum value greater than 1.01:")
        display(cases_gt_1_01.value_counts().sort_index())
else:
    print("No problematic cases found.")
'''

'\nif not problematic_cases.empty:\n    # Filter problematic_cases for sums greater than 1.01\n    cases_gt_1_01 = problematic_cases[problematic_cases > 1.01]\n\n    print(f"Number of cases where condition_weight sum is greater than 1.01: {len(cases_gt_1_01)}")\n    if not cases_gt_1_01.empty:\n        print("\nCounts of each sum value greater than 1.01:")\n        display(cases_gt_1_01.value_counts().sort_index())\nelse:\n    print("No problematic cases found.")\n'

Out of 5033 cases, there are 143 problematic rows where the weight equals to above 1.0

In [ ]:
'''
# Convert stringified lists/dicts to real Python objects
for col in [
    "dermatologist_skin_condition_on_label_name",
    "dermatologist_skin_condition_confidence",
    "weighted_skin_condition_label",
]:
    df_filtered[col] = df_filtered[col].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

# Create a list of weights from the dictionary, aligned with condition names, before exploding
def align_weights(row):
    conditions = row["dermatologist_skin_condition_on_label_name"]
    weights_dict = row["weighted_skin_condition_label"]
    if isinstance(conditions, list) and isinstance(weights_dict, dict):
        return [weights_dict.get(cond) for cond in conditions]
    return [None] * len(conditions) if isinstance(conditions, list) else None # Handle cases where conditions list exists but weights_dict is missing

df_filtered["aligned_condition_weights"] = df_filtered.apply(align_weights, axis=1)

# Now explode all three aligned lists
df_filtered = df_filtered.explode(
    [
        "dermatologist_skin_condition_on_label_name",
        "dermatologist_skin_condition_confidence",
        "aligned_condition_weights",  # Add the new aligned weights column here
    ]
).reset_index(drop=True)

# Rename for clarity
df_filtered = df_filtered.rename(columns={
    "dermatologist_skin_condition_on_label_name": "condition_name",
    "dermatologist_skin_condition_confidence": "condition_confidence",
    "aligned_condition_weights": "condition_weight"  # Rename to the desired final column name
})

# Drop the original dictionary column for weights as it's no longer needed in its original form
df_filtered = df_filtered.drop(columns=["weighted_skin_condition_label"])

# Preview final structure
print(df_filtered[["case_id", "condition_name", "condition_confidence", "condition_weight"]].head(10))
'''

'\n# Convert stringified lists/dicts to real Python objects\nfor col in [\n    "dermatologist_skin_condition_on_label_name",\n    "dermatologist_skin_condition_confidence",\n    "weighted_skin_condition_label",\n]:\n    df_filtered[col] = df_filtered[col].apply(\n        lambda x: ast.literal_eval(x) if isinstance(x, str) else x\n    )\n\n# Create a list of weights from the dictionary, aligned with condition names, before exploding\ndef align_weights(row):\n    conditions = row["dermatologist_skin_condition_on_label_name"]\n    weights_dict = row["weighted_skin_condition_label"]\n    if isinstance(conditions, list) and isinstance(weights_dict, dict):\n        return [weights_dict.get(cond) for cond in conditions]\n    return [None] * len(conditions) if isinstance(conditions, list) else None # Handle cases where conditions list exists but weights_dict is missing\n\ndf_filtered["aligned_condition_weights"] = df_filtered.apply(align_weights, axis=1)\n\n# Now explode all three aligned list

In [ ]:
'''
import os

save_dir = "/content/drive/My Drive/BTT_Skinterest_2A/Dataset"
save_path = os.path.join(save_dir, "dermatologist_conditions_split.csv")
df_filtered.to_csv(save_path, index=False)
'''

'\nimport os\n\nsave_dir = "/content/drive/My Drive/BTT_Skinterest_2A/Dataset"\nsave_path = os.path.join(save_dir, "dermatologist_conditions_split.csv")\ndf_filtered.to_csv(save_path, index=False)\n'

 ## Image Cleaning

In [16]:
df_filtered_path = "/content/drive/My Drive/BTT_Skinterest_2A/Dataset/dermatologist_conditions_split.csv"
df_filtered = pd.read_csv(df_filtered_path)
df_filtered[:10]

/tmp/ipython-input-932086250.py:2: DtypeWarning: Columns (68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  df_filtered = pd.read_csv(df_filtered_path)


,case_id,source,release,year,age_group,sex_at_birth,fitzpatrick_skin_type,race_ethnicity_american_indian_or_alaska_native,race_ethnicity_asian,race_ethnicity_black_or_african_american,...,dermatologist_gradable_for_fitzpatrick_skin_type_2,dermatologist_gradable_for_fitzpatrick_skin_type_3,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,gradable_for_monk_skin_tone_india,gradable_for_monk_skin_tone_us,monk_skin_tone_label_india,monk_skin_tone_label_us,condition_weight
0,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0,0.41
1,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0,0.41
2,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0,0.18
3,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0,0.41
4,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0,0.18
5,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0,0.41
6,-1003358831658393077,SCIN,1.0.0,2023,AGE_18_TO_29,MALE,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST4,NaN,NaN,True,True,3.0,4.0,0.55
7,-1003358831658393077,SCIN,1.0.0,2023,AGE_18_TO_29,MALE,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST4,NaN,NaN,True,True,3.0,4.0,0.23
8,-1003358831658393077,SCIN,1.0.0,2023,AGE_18_TO_29,MALE,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST4,NaN,NaN,True,True,3.0,4.0,0.23
9,-1003844406100696311,SCIN,1.0.0,2023,AGE_40_TO_49,FEMALE,FST3,0.0,0.0,0.0,...,NaN,NaN,FST1,NaN,NaN,True,True,1.0,1.0,0.33


In [17]:
from google.cloud import storage
# List all images in dataset/images/
all_images = []
for blob in Globals.gcs_bucket.list_blobs(prefix=Globals.gcs_images_dir):
    if blob.name.endswith((".jpg",".jpeg",".png")):
        all_images.append(blob.name)

In [18]:
# Problematic images identified by CleanVision
to_drop = [
    "dataset/images/-4593817128438983108.png",
    "dataset/images/-2431769699504014881.png"
]

# Columns that contain image paths
image_cols = ['image_1_path', 'image_2_path', 'image_3_path']

# Build a boolean mask for rows that have any of these images
mask = df_filtered[image_cols].isin(to_drop).any(axis=1)

# Apply filter (keep only rows without those images)
df_filtered = df_filtered[~mask].copy()

In [19]:
save_dir = "/content/drive/My Drive/BTT_Skinterest_2A/Dataset"
os.makedirs(save_dir, exist_ok=True)

In [20]:
save_path = os.path.join(save_dir, "image_quality_report.csv")
quality_df = pd.read_csv(save_path)
print("Loaded", len(quality_df), "rows")
quality_df.head()

Loaded 10379 rows


,image_path,blur,brightness_mean,brightness_std,underexp,overexp,contrast,shadow
0,dataset/images/-1001492676369731180.png,49.800900,125.822289,62.991703,0.152014,0.049442,0.988142,0.162389
1,dataset/images/-1001733364362669777.png,238.390468,74.623044,29.900723,0.036180,0.000554,1.000000,0.036180
2,dataset/images/-1003800477193786941.png,89.631693,128.779480,47.623139,0.005774,0.023419,0.980237,0.055794
3,dataset/images/-1005922060850163675.png,4.172010,102.138025,62.690025,0.179942,0.000000,0.990610,0.010904
4,dataset/images/-1007969568196430462.png,9.460231,147.597770,40.054561,0.000000,0.002477,0.819608,0.107458


In [21]:
# Problematic images identified by CleanVision
to_drop_2 = [
    "dataset/images/4207723573736028617.png",
]

mask_2 = df_filtered[image_cols].isin(to_drop_2).any(axis=1)
df_filtered = df_filtered.loc[~mask_2]

print(f"Removed {mask_2.sum()} rows containing problematic images.")
print(f" Filtered dataset now has {len(df_filtered)} rows.")

Removed 0 rows containing problematic images.
 Filtered dataset now has 8209 rows.


In [22]:
save_path = os.path.join(save_dir, "image_quality_report_with_sharpness.csv")
quality_df = pd.read_csv(save_path)
print("Loaded", len(quality_df), "rows")
quality_df.head()

Loaded 8209 rows


/tmp/ipython-input-2692060160.py:2: DtypeWarning: Columns (69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  quality_df = pd.read_csv(save_path)


,case_id,source,release,year,age_group,sex_at_birth,fitzpatrick_skin_type,race_ethnicity_american_indian_or_alaska_native,race_ethnicity_asian,race_ethnicity_black_or_african_american,...,dermatologist_gradable_for_fitzpatrick_skin_type_1,dermatologist_gradable_for_fitzpatrick_skin_type_2,dermatologist_gradable_for_fitzpatrick_skin_type_3,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,gradable_for_monk_skin_tone_india,gradable_for_monk_skin_tone_us,monk_skin_tone_label_india,monk_skin_tone_label_us
0,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0
1,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0
2,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0
3,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0
4,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0


# Training Dataset defined


In [23]:
df_filtered.columns.tolist()

['case_id',
 'source',
 'release',
 'year',
 'age_group',
 'sex_at_birth',
 'fitzpatrick_skin_type',
 'race_ethnicity_american_indian_or_alaska_native',
 'race_ethnicity_asian',
 'race_ethnicity_black_or_african_american',
 'race_ethnicity_hispanic_latino_or_spanish_origin',
 'race_ethnicity_middle_eastern_or_north_african',
 'race_ethnicity_native_hawaiian_or_pacific_islander',
 'race_ethnicity_white',
 'race_ethnicity_other_race',
 'race_ethnicity_prefer_not_to_answer',
 'textures_raised_or_bumpy',
 'textures_flat',
 'textures_rough_or_flaky',
 'textures_fluid_filled',
 'body_parts_head_or_neck',
 'body_parts_arm',
 'body_parts_palm',
 'body_parts_back_of_hand',
 'body_parts_torso_front',
 'body_parts_torso_back',
 'body_parts_genitalia_or_groin',
 'body_parts_buttocks',
 'body_parts_leg',
 'body_parts_foot_top_or_side',
 'body_parts_foot_sole',
 'body_parts_other',
 'condition_symptoms_bothersome_appearance',
 'condition_symptoms_bleeding',
 'condition_symptoms_increasing_size',
 'c

In [ ]:
'''
df_filtered_path = "/content/drive/My Drive/BTT_Skinterest_2A/Dataset/dermatologist_conditions_split.csv"
df_filtered = pd.read_csv(df_filtered_path)
df_filtered[:10]
'''

'\ndf_filtered_path = "/content/drive/My Drive/BTT_Skinterest_2A/Dataset/dermatologist_conditions_split.csv"\ndf_filtered = pd.read_csv(df_filtered_path)\ndf_filtered[:10]\n'

In [24]:
df_filtered.columns.tolist()

['case_id',
 'source',
 'release',
 'year',
 'age_group',
 'sex_at_birth',
 'fitzpatrick_skin_type',
 'race_ethnicity_american_indian_or_alaska_native',
 'race_ethnicity_asian',
 'race_ethnicity_black_or_african_american',
 'race_ethnicity_hispanic_latino_or_spanish_origin',
 'race_ethnicity_middle_eastern_or_north_african',
 'race_ethnicity_native_hawaiian_or_pacific_islander',
 'race_ethnicity_white',
 'race_ethnicity_other_race',
 'race_ethnicity_prefer_not_to_answer',
 'textures_raised_or_bumpy',
 'textures_flat',
 'textures_rough_or_flaky',
 'textures_fluid_filled',
 'body_parts_head_or_neck',
 'body_parts_arm',
 'body_parts_palm',
 'body_parts_back_of_hand',
 'body_parts_torso_front',
 'body_parts_torso_back',
 'body_parts_genitalia_or_groin',
 'body_parts_buttocks',
 'body_parts_leg',
 'body_parts_foot_top_or_side',
 'body_parts_foot_sole',
 'body_parts_other',
 'condition_symptoms_bothersome_appearance',
 'condition_symptoms_bleeding',
 'condition_symptoms_increasing_size',
 'c

In [25]:
df_filtered[['condition_symptoms_bothersome_appearance',
 'condition_symptoms_bleeding',
 'condition_symptoms_increasing_size',
 'condition_symptoms_darkening',
 'condition_symptoms_itching',
 'condition_symptoms_burning',
 'condition_symptoms_pain',
 'condition_symptoms_no_relevant_experience',
 'gradable_for_monk_skin_tone_india',
 'gradable_for_monk_skin_tone_us',
 'monk_skin_tone_label_india',
 'monk_skin_tone_label_us',
]].isnull().sum()

,0
condition_symptoms_bothersome_appearance,0
condition_symptoms_bleeding,0
condition_symptoms_increasing_size,0
condition_symptoms_darkening,0
condition_symptoms_itching,0
condition_symptoms_burning,0
condition_symptoms_pain,0
condition_symptoms_no_relevant_experience,0
gradable_for_monk_skin_tone_india,16
gradable_for_monk_skin_tone_us,16


In [26]:
# Adjust for a few missing values
mode_gradable_india = df_filtered['gradable_for_monk_skin_tone_india'].mode()[0]
mode_gradable_us = df_filtered['gradable_for_monk_skin_tone_us'].mode()[0]
mode_monk_india = df_filtered['monk_skin_tone_label_india'].mode()[0]
mode_monk_us = df_filtered['monk_skin_tone_label_us'].mode()[0]

df_filtered['gradable_for_monk_skin_tone_india'].fillna(mode_gradable_india, inplace=True)
df_filtered['gradable_for_monk_skin_tone_us'].fillna(mode_gradable_us, inplace=True)
df_filtered['monk_skin_tone_label_india'].fillna(mode_monk_india, inplace=True)
df_filtered['monk_skin_tone_label_us'].fillna(mode_monk_us, inplace=True)


/tmp/ipython-input-3646964942.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filtered['gradable_for_monk_skin_tone_india'].fillna(mode_gradable_india, inplace=True)
/tmp/ipython-input-3646964942.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filtered['gradable_for_monk_skin_tone_india'].fillna(m

In [27]:
df_filtered[['condition_symptoms_bothersome_appearance',
 'condition_symptoms_bleeding',
 'condition_symptoms_increasing_size',
 'condition_symptoms_darkening',
 'condition_symptoms_itching',
 'condition_symptoms_burning',
 'condition_symptoms_pain',
 'condition_symptoms_no_relevant_experience',
 'gradable_for_monk_skin_tone_india',
 'gradable_for_monk_skin_tone_us',
 'monk_skin_tone_label_india',
 'monk_skin_tone_label_us',
]].isnull().sum()

,0
condition_symptoms_bothersome_appearance,0
condition_symptoms_bleeding,0
condition_symptoms_increasing_size,0
condition_symptoms_darkening,0
condition_symptoms_itching,0
condition_symptoms_burning,0
condition_symptoms_pain,0
condition_symptoms_no_relevant_experience,0
gradable_for_monk_skin_tone_india,0
gradable_for_monk_skin_tone_us,0


## Baseline Model

In [28]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [29]:
# Select highest-confidence condition per case
train_df = df_filtered.copy()

train_df = train_df.loc[train_df.groupby("case_id")["condition_confidence"].idxmax()]
train_df = train_df.reset_index(drop=True)

# Assign top label
train_df["top_label"] = train_df["condition_name"]


In [30]:
print(train_df.head())
print(df_filtered.shape)
print(train_df.shape)

               case_id source release  year     age_group  \
0 -9223126124516060635   SCIN   1.0.0  2023  AGE_18_TO_29   
1 -9221035133244224964   SCIN   1.0.0  2023   AGE_UNKNOWN   
2 -9218285092875997194   SCIN   1.0.0  2023  AGE_30_TO_39   
3 -9215921701035518354   SCIN   1.0.0  2023   AGE_UNKNOWN   
4 -9212359218854294582   SCIN   1.0.0  2023   AGE_UNKNOWN   

           sex_at_birth fitzpatrick_skin_type  \
0                FEMALE                  FST2   
1  OTHER_OR_UNSPECIFIED       NONE_IDENTIFIED   
2                  MALE       NONE_IDENTIFIED   
3  OTHER_OR_UNSPECIFIED       NONE_IDENTIFIED   
4  OTHER_OR_UNSPECIFIED       NONE_IDENTIFIED   

   race_ethnicity_american_indian_or_alaska_native  race_ethnicity_asian  \
0                                              0.0                   0.0   
1                                              0.0                   0.0   
2                                              0.0                   0.0   
3                                 

In [31]:
image_cols = ["image_1_path", "image_2_path", "image_3_path"]

df_long = train_df.melt(
    id_vars=["case_id", "top_label"],
    value_vars=image_cols,
    var_name="image_num",
    value_name="image_path"
)

# Drop rows with missing image paths
df_long = df_long.dropna(subset=["image_path"])


In [32]:
df_long["image_path"] = "gs://" + Globals.gcs_bucket_name + "/" + df_long["image_path"].astype(str)

In [33]:
le = LabelEncoder()
df_long["label_id"] = le.fit_transform(df_long["top_label"])
num_classes = len(le.classes_)

print(f"Found {num_classes} unique labels.")


Found 210 unique labels.


In [34]:
df_long.head()

,case_id,top_label,image_num,image_path,label_id
0,-9223126124516060635,Scabies,image_1_path,gs://dx-scin-public-data/dataset/images/342227...,176
1,-9221035133244224964,Eczema,image_1_path,gs://dx-scin-public-data/dataset/images/-31623...,56
2,-9218285092875997194,Irritant Contact Dermatitis,image_1_path,gs://dx-scin-public-data/dataset/images/-40639...,99
3,-9215921701035518354,Tinea,image_1_path,gs://dx-scin-public-data/dataset/images/-15900...,190
4,-9212359218854294582,Eczema,image_1_path,gs://dx-scin-public-data/dataset/images/-85001...,56


In [35]:
# Remove classes that have fewer than 2 images
label_counts = df_long["label_id"].value_counts()
df_long = df_long[df_long["label_id"].isin(label_counts[label_counts >= 2].index)]

In [36]:
label_counts

,count
label_id,
56,1042
19,546
196,442
96,392
68,301
...,...
55,1
110,1
164,1


In [37]:
# First compute existence for ALL rows
df_long["file_exists"] = df_long["image_path"].apply(tf.io.gfile.exists)

# Extract invalid paths
invalid_paths = df_long[df_long["file_exists"] == False]["image_path"].tolist()

print("Number of invalid images:", len(invalid_paths))
invalid_paths[:20]   # preview first 20


Number of invalid images: 1


['gs://dx-scin-public-data/dataset/images/-2243186711511406658.png']

In [38]:
df_long = df_long[df_long["file_exists"] == True].drop(columns=["file_exists"])

In [39]:
paths = df_long["image_path"].values
labels = df_long["label_id"].values


train_paths, test_paths, train_labels, test_labels = train_test_split(
    paths, labels,
    test_size=0.4,
    random_state=42,
    stratify=labels
)


In [40]:
IMG_SIZE = (224, 224)

def load_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.cast(label, tf.int32)   # IMPORTANT
    return image, label

In [41]:
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_ds = test_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)


In [42]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [43]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)

Epoch 1/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 95s 592ms/step - accuracy: 0.1080 - loss: 8.7192 - val_accuracy: 0.1595 - val_loss: 3.9417
Epoch 2/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 64s 439ms/step - accuracy: 0.1739 - loss: 3.7661 - val_accuracy: 0.1572 - val_loss: 3.9264
Epoch 3/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 57s 391ms/step - accuracy: 0.3057 - loss: 3.1238 - val_accuracy: 0.1322 - val_loss: 4.0716
Epoch 4/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 52s 352ms/step - accuracy: 0.5241 - loss: 2.1546 - val_accuracy: 0.1206 - val_loss: 4.4381
Epoch 5/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 48s 326ms/step - accuracy: 0.7134 - loss: 1.3630 - val_accuracy: 0.0998 - val_loss: 5.0389
Epoch 6/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 47s 323ms/step - accuracy: 0.8296 - loss: 0.8572 - val_accuracy: 0.0913 - val_loss: 5.4087
Epoch 7/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 46s 319ms/step - accuracy: 0.9138 - loss: 0.4717 - val_accuracy: 0.1052 - val_loss: 6.2691
Epoch 8/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 46s 312ms/step - accuracy: 0.9624 - loss: 0

In [44]:
#Save model
model_save_dir = '/content/drive/My Drive/BTT_Skinterest_2A/Coding for Skinterest Tech 2A Project/Nivi_Models'
os.makedirs(model_save_dir, exist_ok=True)
model_path = os.path.join(model_save_dir, 'cnn_dermatology_model.keras')
model.save(model_path)

In [46]:
#Load model
loaded_model = tf.keras.models.load_model("/content/drive/My Drive/BTT_Skinterest_2A/Coding for Skinterest Tech 2A Project/Nivi_Models/cnn_dermatology_model.keras")

In [49]:
test_loss, test_accuracy = model.evaluate(test_ds, verbose=1)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

82/82 ━━━━━━━━━━━━━━━━━━━━ 17s 210ms/step - accuracy: 0.0958 - loss: 7.6180
Test Loss: 7.6039
Test Accuracy: 0.1025


In [50]:
train_loss, train_accuracy = model.evaluate(train_ds, verbose=1)
print(f'Train Loss: {train_loss:.4f}')
print(f'Train Accuracy: {train_accuracy:.4f}')

122/122 ━━━━━━━━━━━━━━━━━━━━ 29s 175ms/step - accuracy: 0.9948 - loss: 0.0595
Train Loss: 0.0621
Train Accuracy: 0.9951


In [51]:
# Save label encoder
import pickle
le_save_dir = '/content/drive/My Drive/BTT_Skinterest_2A/Coding for Skinterest Tech 2A Project/Nivi_Models'
os.makedirs(le_save_dir, exist_ok=True)
le_path = os.path.join(le_save_dir, 'label_encoder_baseline.pkl')
with open(le_path, "wb") as f:
    pickle.dump(le, f)


In [52]:
# Load pickle encoder
import pickle

with open(le_path, "rb") as f:
    le = pickle.load(f)


## Model with Condition Symptoms

In [53]:
condition_symptoms_cols = [
 'condition_symptoms_bothersome_appearance',
 'condition_symptoms_bleeding',
 'condition_symptoms_increasing_size',
 'condition_symptoms_darkening',
 'condition_symptoms_itching',
 'condition_symptoms_burning',
 'condition_symptoms_pain',
 'condition_symptoms_no_relevant_experience',
]

# extract case_id + condition symptom columns
df_condition_symptoms = train_df[['case_id'] + condition_symptoms_cols].drop_duplicates()

# merge condition symptoms metadata onto df_long rows
df_long = df_long.merge(df_condition_symptoms, on='case_id', how='left')
df_long

,case_id,top_label,image_num,image_path,label_id,condition_symptoms_bothersome_appearance,condition_symptoms_bleeding,condition_symptoms_increasing_size,condition_symptoms_darkening,condition_symptoms_itching,condition_symptoms_burning,condition_symptoms_pain,condition_symptoms_no_relevant_experience
0,-9223126124516060635,Scabies,image_1_path,gs://dx-scin-public-data/dataset/images/342227...,176,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-9221035133244224964,Eczema,image_1_path,gs://dx-scin-public-data/dataset/images/-31623...,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-9218285092875997194,Irritant Contact Dermatitis,image_1_path,gs://dx-scin-public-data/dataset/images/-40639...,99,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,-9215921701035518354,Tinea,image_1_path,gs://dx-scin-public-data/dataset/images/-15900...,190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-9212359218854294582,Eczema,image_1_path,gs://dx-scin-public-data/dataset/images/-85001...,56,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6481,9114324332147869982,Allergic Contact Dermatitis,image_3_path,gs://dx-scin-public-data/dataset/images/410063...,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6482,9114737540227929305,Eczema,image_3_path,gs://dx-scin-public-data/dataset/images/-98950...,56,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6483,9130094661402000696,Acanthosis nigricans,image_3_path,gs://dx-scin-public-data/dataset/images/858131...,4,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
6484,9138173567729098672,Acute and chronic dermatitis,image_3_path,gs://dx-scin-public-data/dataset/images/267391...,11,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [54]:
paths = df_long["image_path"].values
labels = df_long["label_id"].values
metadata = df_long[condition_symptoms_cols].astype("float32").values

In [55]:
df_long["file_exists"] = df_long["image_path"].apply(tf.io.gfile.exists)
invalid_paths = df_long[df_long["file_exists"] == False]["image_path"].tolist()

print("Number of invalid images:", len(invalid_paths))
invalid_paths[:20]


Number of invalid images: 0


[]

In [56]:
df_long = df_long[df_long["file_exists"] == True].drop(columns=["file_exists"])

In [57]:
# Remove classes that have fewer than 2 images
label_counts = df_long["label_id"].value_counts()
df_long = df_long[df_long["label_id"].isin(label_counts[label_counts >= 2].index)]

In [58]:
train_p, test_p, train_l, test_l, train_m, test_m = train_test_split(
    paths, labels, metadata,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

IMG_SIZE = (224, 224)

def load_image_with_meta(path, label, meta):
    img_bytes = tf.io.read_file(path)
    img = tf.image.decode_png(img_bytes, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    return (img, meta), label


In [59]:
train_ds = tf.data.Dataset.from_tensor_slices((train_p, train_l, train_m))
train_ds = train_ds.map(load_image_with_meta, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_p, test_l, test_m))
test_ds = test_ds.map(load_image_with_meta, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)


In [60]:
image_input = layers.Input(shape=(224, 224, 3))
x = layers.Conv2D(32, 3, activation='relu')(image_input)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)

meta_input = layers.Input(shape=(len(condition_symptoms_cols),))
m = layers.Dense(32, activation='relu')(meta_input)

combined = layers.concatenate([x, m])
output = layers.Dense(num_classes, activation='softmax')(combined)

model_with_condition_symptoms = models.Model(inputs=[image_input, meta_input], outputs=output)
model_with_condition_symptoms.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [61]:
model_with_condition_symptoms.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 58s 273ms/step - accuracy: 0.1227 - loss: 4.6479 - val_accuracy: 0.1602 - val_loss: 3.8640
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 48s 248ms/step - accuracy: 0.1753 - loss: 3.7482 - val_accuracy: 0.1418 - val_loss: 3.8071
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 45s 232ms/step - accuracy: 0.2560 - loss: 3.2311 - val_accuracy: 0.1410 - val_loss: 4.0983
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 45s 230ms/step - accuracy: 0.4566 - loss: 2.2910 - val_accuracy: 0.1194 - val_loss: 5.3910
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 45s 232ms/step - accuracy: 0.6614 - loss: 1.3838 - val_accuracy: 0.1102 - val_loss: 7.4178
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 47s 242ms/step - accuracy: 0.8215 - loss: 0.7656 - val_accuracy: 0.0871 - val_loss: 9.8687
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 53s 274ms/step - accuracy: 0.8982 - loss: 0.4427 - val_accuracy: 0.0901 - val_loss: 11.5857
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 47s 239ms/step - accuracy: 0.9149 - loss: 

In [62]:
#Save model
model_save_dir = "/content/drive/My Drive/BTT_Skinterest_2A/Coding for Skinterest Tech 2A Project/Nivi_Models"
os.makedirs(model_save_dir, exist_ok=True)
model_path_condition_symptoms = os.path.join(model_save_dir, 'cnn_dermatology_model_condition_symptoms.keras')
model.save(model_path_condition_symptoms)
#model.save("cnn_dermatology_model_condition_symptoms.keras")

In [ ]:
#import pickle
# used the label encodings for the dermatologist conditions from the baseline model

#with open(f"label_encoder_condition_symptoms.pkl", "wb") as f:
#    pickle.dump(le, f)


## Model with Monk Skin Tone Scores

In [ ]:
'''
monk_cols = [
 'gradable_for_monk_skin_tone_india',
 'gradable_for_monk_skin_tone_us',
 'monk_skin_tone_label_india',
 'monk_skin_tone_label_us'
]

# pick top condition per case
base_df = df_filtered.copy()
base_df = base_df.loc[base_df.groupby("case_id")["condition_confidence"].idxmax()]
base_df = base_df.reset_index(drop=True)
base_df["top_label"] = base_df["condition_name"]

# melt images
df_long = base_df.melt(
    id_vars=["case_id", "top_label"],
    value_vars=image_cols,
    var_name="image_num",
    value_name="image_path"
)
df_long = df_long.dropna(subset=["image_path"])

# build GCS path
df_long["image_path"] = (
    "gs://" + Globals.gcs_bucket_name + "/" + df_long["image_path"].astype(str).str.strip()
)

# add monk scale metadata
df_monk_cols = base_df[['case_id'] + monk_cols].drop_duplicates()
df_long = df_long.merge(df_monk_cols, on="case_id", how="left")

# label encode
le = LabelEncoder()
df_long["label_id"] = le.fit_transform(df_long["top_label"])
num_classes = len(le.classes_)

# remove invalid image paths
df_long["file_exists"] = df_long["image_path"].apply(tf.io.gfile.exists)
df_long = df_long[df_long["file_exists"]].drop(columns=["file_exists"])

# remove labels with <2 samples
counts = df_long["label_id"].value_counts()
valid_labels = counts[counts >= 2].index
df_long = df_long[df_long["label_id"].isin(valid_labels)]
num_classes = df_long["label_id"].nunique()

# build arrays
paths = df_long["image_path"].values
labels = df_long["label_id"].values
meta = df_long[monk_cols].astype("float32").values

# split
train_p, test_p, train_l, test_l, train_m, test_m = train_test_split(
    paths, labels, meta,
    test_size=0.4,
    random_state=42,
    stratify=labels
)

# loader
def load_image_with_meta(path, label, meta_vec):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = img / 255.0
    return (img, meta_vec), tf.cast(label, tf.int32)

# datasets
train_ds = (
    tf.data.Dataset.from_tensor_slices((train_p, train_l, train_m))
    .map(load_image_with_meta, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(1000)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

test_ds = (
    tf.data.Dataset.from_tensor_slices((test_p, test_l, test_m))
    .map(load_image_with_meta, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

# CNN image branch
image_input = layers.Input(shape=(224, 224, 3))
x = layers.Conv2D(32, 3, activation='relu')(image_input)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)

# Monk scale branch
meta_input = layers.Input(shape=(len(monk_cols),))
m = layers.Dense(16, activation='relu')(meta_input)
m = layers.Dense(8, activation='relu')(m)

# combine the branches
combined = layers.concatenate([x, m])
output = layers.Dense(num_classes, activation='softmax')(combined)

model = models.Model(inputs=[image_input, meta_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train
history = model.fit(train_ds, validation_data=test_ds, epochs=10)

# Save Model
model_save_dir = "/content/drive/My Drive/BTT_Skinterest_2A/Coding for Skinterest Tech 2A Project/Nivi_Models"
os.makedirs(model_save_dir, exist_ok=True)
model_path_monk_scale = os.path.join(model_save_dir, 'cnn_dermatology_model_monk_scale.keras')
model.save(model_path_monk_scale)
#model.save("cnn_dermatology_monk_scale_model.keras")
'''

In [63]:
monk_cols = [
 'gradable_for_monk_skin_tone_india',
 'gradable_for_monk_skin_tone_us',
 'monk_skin_tone_label_india',
 'monk_skin_tone_label_us'
]

# add monk scale metadata
df_monk_cols = train_df[['case_id'] + monk_cols].drop_duplicates()
df_long = df_long.merge(df_monk_cols, on="case_id", how="left")

# build arrays
paths = df_long["image_path"].values
labels = df_long["label_id"].values
meta = df_long[monk_cols].astype("float32").values

# split
train_p, test_p, train_l, test_l, train_m, test_m = train_test_split(
    paths, labels, meta,
    test_size=0.4,
    random_state=42,
    stratify=labels
)

# loader
def load_image_with_meta(path, label, meta_vec):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = img / 255.0
    return (img, meta_vec), tf.cast(label, tf.int32)

# datasets
train_ds = (
    tf.data.Dataset.from_tensor_slices((train_p, train_l, train_m))
    .map(load_image_with_meta, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(1000)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

test_ds = (
    tf.data.Dataset.from_tensor_slices((test_p, test_l, test_m))
    .map(load_image_with_meta, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

# CNN image branch
image_input = layers.Input(shape=(224, 224, 3))
x = layers.Conv2D(32, 3, activation='relu')(image_input)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
'''
# Gradable Monk scale branch (true/false values)
meta_gradable_input = layers.Input(shape=(len(monk_cols[:2]),))
g = layers.Dense(16, activation='relu')(meta_gradable_input)
g = layers.Dense(8, activation='relu')(g)

# Monk scale branch (numerical values)
meta_monk_input = layers.Input(shape=(len(monk_cols[2:4]),))
m = layers.Dense(16, activation='relu')(meta_monk_input)
m = layers.Dense(8, activation='relu')(m)
'''
# Monk scale branch
meta_monk_input = layers.Input(shape=(len(monk_cols),))
m = layers.Dense(16, activation='relu')(meta_monk_input)
m = layers.Dense(8, activation='relu')(m)

# Combine the branches
combined = layers.concatenate([x, m])
output = layers.Dense(num_classes, activation='softmax')(combined)

model_monk_scale = models.Model(inputs=[image_input, meta_monk_input], outputs=output)
model_monk_scale.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [64]:
model.fit(train_ds, validation_data=test_ds, epochs=10)

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=('Tensor(shape=(None, 224, 224, 3))', 'Tensor(shape=(None, 4))')
  warnings.warn(msg)


122/122 ━━━━━━━━━━━━━━━━━━━━ 61s 405ms/step - accuracy: 0.9932 - loss: 0.0655 - val_accuracy: 0.0863 - val_loss: 7.7152
Epoch 2/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 52s 354ms/step - accuracy: 0.9967 - loss: 0.0426 - val_accuracy: 0.0956 - val_loss: 8.2826
Epoch 3/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 49s 331ms/step - accuracy: 0.9981 - loss: 0.0230 - val_accuracy: 0.0890 - val_loss: 8.3318
Epoch 4/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 47s 320ms/step - accuracy: 0.9987 - loss: 0.0173 - val_accuracy: 0.0879 - val_loss: 8.7060
Epoch 5/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 48s 334ms/step - accuracy: 0.9997 - loss: 0.0112 - val_accuracy: 0.0960 - val_loss: 8.8431
Epoch 6/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 47s 319ms/step - accuracy: 0.9993 - loss: 0.0088 - val_accuracy: 0.0929 - val_loss: 8.8951
Epoch 7/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 49s 331ms/step - accuracy: 0.9999 - loss: 0.0048 - val_accuracy: 0.0983 - val_loss: 9.1882
Epoch 8/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 45s 310ms/step - accuracy: 0.9997 - loss: 0.0046 - val

In [65]:
model_save_dir = "/content/drive/My Drive/BTT_Skinterest_2A/Coding for Skinterest Tech 2A Project/Nivi_Models"
os.makedirs(model_save_dir, exist_ok=True)
model_path_monk_scale = os.path.join(model_save_dir, 'cnn_dermatology_model_monk_scale.keras')
model.save(model_path_monk_scale)
#model.save("cnn_dermatology_monk_scale_model.keras")